### Joel Brigida

### CDA4321: Cryptographic Engineering

### Lecture 5: Code for Assignment 2

### Multi-Precision Arithmetic:
 - If processor word size = 64-bit
     - Split a 256-bit number into a 4 index array of 64-bit numbers
 - If processor word size = 32-bit
     - Split a 256-but number into an 8 index array of 32-bit numbers

### $ res[0] = a[0] + b[0] $
### $ res[1] = a[1] + b[1] + \text{carry from } res[0] $
### $ res[2] = a[2] + b[2] + \text{carry from } res[1] $
### $ ... $

 - For a 256-bit number & 64-bit word size: use an array with 8 64-bit indices (Total allocation = 512 bits).
     - Every even index will contain 0's and used only to deal with the carry

In [1]:
result = 0x1234 - 0x4321
result2 = 0x1234 + (-0x4321)
print(f'{hex(0x1234)} - {hex(0x4321)} = {hex(result)}\n')
print(result.bits(), '\n')
print(hex(result2), '\n')
print(result.bits(), '\n')
bitList = list(result.bits())
print(bitList)

revBitList = bitList[::-1]
print(revBitList)

0x1234 - 0x4321 = -0x30ed

[-1, 0, -1, -1, 0, -1, -1, -1, 0, 0, 0, 0, -1, -1] 

-0x30ed 

[-1, 0, -1, -1, 0, -1, -1, -1, 0, 0, 0, 0, -1, -1] 

[-1, 0, -1, -1, 0, -1, -1, -1, 0, 0, 0, 0, -1, -1]
[-1, -1, 0, 0, 0, 0, -1, -1, -1, 0, -1, -1, 0, -1]


In [2]:
result3 = 0xFFFFCF13

print(result3.bits(), ' MSB on Right\n')
bitlist2 = result3.bits()
revBitList2 = bitlist2[::-1]
print(revBitList2, ' MSB on Left\n')

[1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  MSB on Right

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1]  MSB on Left



In [3]:
a = 812937419827341987234918723501982375897410237 # A very large number
d = 4 # base 2^d: octal = 2^3 = 8, hex = 2^4 = 16
s = 4
print(a.digits(2^d), '\n') # print in octal or hex format

n = 4 # number of hex digits in each array index
hexArray = ["0x{0:0{1}x}".format(x,n) for x in a.digits(base=2^d,padto=s)]
print(hexArray, '\n')

v = 0xFFF7C3ED66
print(Integer(v))

[13, 11, 10, 2, 11, 0, 2, 10, 10, 4, 7, 6, 6, 14, 5, 3, 9, 7, 10, 5, 5, 12, 14, 5, 11, 6, 6, 13, 7, 8, 12, 5, 0, 1, 4, 7, 4, 2] 

['0x000d', '0x000b', '0x000a', '0x0002', '0x000b', '0x0000', '0x0002', '0x000a', '0x000a', '0x0004', '0x0007', '0x0006', '0x0006', '0x000e', '0x0005', '0x0003', '0x0009', '0x0007', '0x000a', '0x0005', '0x0005', '0x000c', '0x000e', '0x0005', '0x000b', '0x0006', '0x0006', '0x000d', '0x0007', '0x0008', '0x000c', '0x0005', '0x0000', '0x0001', '0x0004', '0x0007', '0x0004', '0x0002'] 

1099373473126


Source: https://nbviewer.org/urls/pastebin.com/raw/GJMCtHfA

Modifications Mine.

In [4]:
DATA_TYPE=32
def addn(a, b, r=DATA_TYPE): return (a + b) % (2^r)
def subn(a, b, r=DATA_TYPE): return (a - b) % (2^r)
def muln(a, b, r=DATA_TYPE): return (a * b) % (2^r)
def shift_leftn(a, b, r=DATA_TYPE):return (a >> b) % (2^r)
def shift_rightn(a, b, r=DATA_TYPE):return (a << b) % (2^r)
def andn(a, b, r=DATA_TYPE): return (a & b) % (2^r)
def orn(a, b, r=DATA_TYPE): return (a & b) % (2^r)
def xorn(a, b, r=DATA_TYPE): return (a.xor(b)) % (2^r)
def notn(a, r=DATA_TYPE): return (~a) % (2^r)

print(hex(subn(0x0, 0x2)), '\n')
print(hex(subn(0x1234, 0x4321)), '\n')

def print_integer_hex(a, base, padto, digits, s = ""):
    print(s + "[" + ','.join(["0x{0:0{1}x}".format(x,digits) for x in a.digits(base=base,padto=padto)]) + "]")

a = 0x123574fa
print_integer_hex(a, 2^8, 5, 4, "a = ")

0xfffffffe 

0xffffcf13 

a = [0x00fa,0x0074,0x0035,0x0012,0x0000]
